<a href="https://colab.research.google.com/github/taeheee-kim/deep_learning_practice/blob/master/%EC%A0%84%EC%B2%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%EB%8F%8C%EB%A0%A4%EB%B4%85%EC%89%AC%EB%8B%B9%20%EB%81%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Oct  6 23:24:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     |████████████████████████████████| 102 kB 8.5 MB/s 
     |████████████████████████████████| 37.1 MB 81 kB/s 
albumentations==1.1.0 is successfully installed


## Run the example

### Import the required libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
 from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [5]:
import gc
import torch
import torch.utils.data as Data
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


# 데이터 경로
data_path = '/content/drive/My Drive/Tomato/'

train = pd.read_csv(data_path + 'train.csv')
#test = pd.read_csv(data_path + 'test.csv')
#submission = pd.read_csv(data_path + 'sample_submission.csv')

import torch # 파이토치
import random
import os


In [6]:

# 시드 값 고정
seed = 10
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(train,
                                test_size=0.1,
                                stratify=train[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']],
                                random_state=10)

import cv2
from torch.utils.data import Dataset  # 데이터 생성을 위한 클래스
import numpy as np
import glob


class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()  # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    # 데이터 세트 크기 반환 메서드
    def __len__(self):
        return len(self.df)

    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + str(img_id) + '.png' # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image=image)['image']


        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃 값(label)도 함께 반환
        if self.is_test:
            return image
        else:
            # 타깃 값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:11])
            return image, label

In [7]:
# 이미지 변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(400, 600), # 이미지 크기 조절
    # 밝기 및 대비 조절 
    A.RandomBrightnessContrast(brightness_limit=0.1, 
                               contrast_limit=0.1, p=0.5),
    A.VerticalFlip(p=0.5), # 상하 대칭 변환
    A.HorizontalFlip(p=0.5), # 좌우 대칭 변환
    # 이동, 스케일링, 회전 변환 
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=25, p=0.7),
    # 양각화, 날카로움, 블러 효과 
    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.5),
    A.PiecewiseAffine(p=0.5), # 어파인 변환 
    A.Normalize(), # 정규화 변환 
    ToTensorV2() # 텐서로 변환
])

# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(400, 600),
    A.Normalize(),
    ToTensorV2()
])

In [8]:
img_dir = '/content/drive/My Drive/Tomato/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [9]:
def seed_worker(worker_id): #시드를 고정해주는 역할
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [10]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

batch_size = 6

loader_train = DataLoader(dataset_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g)

In [11]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=f47ef40a07fcde86db38c543d2f02190345e0f9edaa0ee47cea6c27fe5603c1b
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [12]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 68.7 MB/s 
     |████████████████████████████████| 56 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 636 kB 65.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모델

# 사전 훈련된 efficientnet-b6 모델 불러오기

model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) 

#model = model.to(device) # 장비 할당

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b6


In [14]:
#모델 이어서 돌리는 코드
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

import copy
PATH = '/content/drive/MyDrive/Tomato/태희/'

#model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) #하시던거 똑같이 유지
device = 'cuda'
model.load_state_dict(copy.deepcopy(torch.load(PATH + "epoch10.pt",device))) # 9, 14, 19, 24, 29 

model = model.to(device) # 장비 할당

#batch_size = 6 #하시던거 똑같이 유지

In [15]:
import torch.nn as nn # 신경망 모듈

# 손실함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00007, weight_decay=0.0001)

from transformers import get_cosine_schedule_with_warmup

epochs = 10 # 총 에폭

# 스케줄러
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*5, 
                                            num_training_steps=len(loader_train)*epochs)


In [16]:
train_loss = []
valid_loss = []
train_acc = []
val_acc = []

In [17]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

# 총 에폭만큼 반복
for epoch in range(epochs):
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
    epoch_is = epoch+11

    # '반복 횟수'만큼 반복 
    for images, labels in tqdm(loader_train):
    
        # 이미지, 레이블(타깃 값) 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가 (훈련 데이터용)
        epoch_train_loss += loss.item() 
        # 역전파 수행
        loss.backward() 
        # 가중치 갱신
        optimizer.step() 
        # 스케줄러 학습률 갱신
        scheduler.step() 
        
    if (epoch_is)%2==0:
          path = '/content/drive/My Drive/Tomato/태희/epoch수정' + str(epoch_is) + '.pt'
          torch.save(model.state_dict(), path)

    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch_is}/{epochs+10}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    model.eval() # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용) 
    preds_list = [] # 예측 확률값 저장용 리스트 초기화 
    true_onehot_list = [] # 실제 타깃 값 저장용 리스트 초기화 
    
    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
            true_onehot = torch.eye(10)[labels].cpu().numpy() # 실제값 (원-핫 인코딩 형식)
            # 예측 확률값과 실제값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
            
        # 검증 데이터 손실값 및 ROC AUC 점수 출력 
        print(f'에폭 [{epoch_is}/{epochs+10}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')  


  0%|          | 0/2079 [00:00<?, ?it/s]

에폭 [11/20] - 훈련 데이터 손실값 : 0.0161
에폭 [11/20] - 검증 데이터 손실값 : 0.0211 / 검증 데이터 ROC AUC : 0.9999


  0%|          | 0/2079 [00:00<?, ?it/s]

에폭 [12/20] - 훈련 데이터 손실값 : 0.0108
에폭 [12/20] - 검증 데이터 손실값 : 0.0235 / 검증 데이터 ROC AUC : 0.9999


  0%|          | 0/2079 [00:00<?, ?it/s]

...

에폭 [18/20] - 훈련 데이터 손실값 : 0.0058
에폭 [18/20] - 검증 데이터 손실값 : 0.0426 / 검증 데이터 ROC AUC : 0.9989


  0%|          | 0/2079 [00:00<?, ?it/s]

에폭 [19/20] - 훈련 데이터 손실값 : 0.0022
에폭 [19/20] - 검증 데이터 손실값 : 0.0545 / 검증 데이터 ROC AUC : 0.9974


  0%|          | 0/2079 [00:00<?, ?it/s]

에폭 [20/20] - 훈련 데이터 손실값 : 0.0027
에폭 [20/20] - 검증 데이터 손실값 : 0.0482 / 검증 데이터 ROC AUC : 0.9979


In [18]:
#PATH = '/content/drive/My Drive/Tomato_version2/'

#모델 저장하기
#torch.save(model, PATH + 'model.pt')

In [19]:
#테스트 사진으로 평가 후 csv 채워넣기

In [20]:
#모델 이어서 돌리는 코드

#import copy
#PATH = '/content/drive/MyDrive/Tomato_version2/태희/'

#model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) #하시던거 똑같이 유지
#device = 'cuda'
#model.load_state_dict(copy.deepcopy(torch.load(PATH + "epoch29.pt",device))) # 9, 14, 19, 24, 29 

#model = model.to(device) # 장비 할당

#batch_size = 6 #하시던거 똑같이 유지

In [21]:
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [22]:
img_dir = '/content/drive/My Drive/Tomato/test_images/'

dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, 
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g)

In [ ]:
num_TTA = 5 # TTA 횟수

preds_tta = np.zeros((len(test), 10)) # 예측 값 저장용 배열 초기화 (TTA용)

# TTA를 적용해 예측 (5에폭)
for i in range(num_TTA):
    print(i)
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            # 타깃 예측 확률
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

0
1
2
3
4


In [ ]:
preds_tta /= num_TTA 

In [ ]:
submission_tta = submission.copy() 

submission_tta[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']] = preds_tta

In [ ]:
submission_tta

In [ ]:
id = submission_tta["image_id"]

In [ ]:
pred_idx = np.argmax(preds_tta, axis=1)  #확률 중 가장 높은 값의 인덱스를 0 ~ 9까지 반환

In [ ]:
pred_idx

In [ ]:
df = pd.DataFrame({'file_name':id, 'answer':pred_idx})

In [ ]:
df 

In [ ]:
df.to_csv('/content/drive/My Drive/Tomato/태희/submission_tta.csv', index=False)